This file filters through all the clean EEBO CSV files to create subfolders of texts. 

Based in part on the 2021 Data+ team's text filtering code: https://github.com/ABeeShake/Ethical-Consumption-Before-Capitalism/blob/main/topic%20modelling/code_actualrunning_awsvm.py

In [ ]:
import os
import re
import pandas as pd
import string

directory = r'C:\Users\amycw\Desktop\ecbc research'
eebofolder = directory+"\\"+'eebo_clean'
eicFolder = directory+"\\"+'eic_eebo'
restorationFolder = directory+"\\"+'restoration_eebo'
monopolyFolder = directory + "\\" + 'monopoly_eebo'

In [ ]:
# move all relevant texts to a separate folder as individual CSVs 
def filterTime(folder, eebofolder, start, end):
    dates = [] 
    count = 0
    for cleanFile in os.listdir(eebofolder):
        readFile = pd.read_csv(eebofolder + "\\" + cleanFile) 
        for i in range(len(readFile.index)):
            data = readFile[i:(i+1)]
            curr_file_name = (cleanFile.rsplit('.', 1)[0]) +'_'+str(i) + ('.csv')
            name = folder + "\\" + curr_file_name
            if isinstance(data.iloc[0].date,str):
                date = (data.iloc[0].date)
            else:
                date = (data.iloc[0].date).astype(str)
            if ((date !='Date Not Found') & (date !='[\'Date Not Found\']')):
                if (date and date.strip()):
                    datenum = re.search('\d{4}', date)
                    if (datenum != None):
                        dates.append(datenum)
                        if (int(str(datenum.group(0))) in range(start, end)):
                            data.to_csv(name, index=False)
            count += 1
            if (count % 10000 == 0):
                print(count) 
filterTime(restorationFolder, eebofolder, 1660,1714)

In [ ]:
#code for adding all DATED EIC texts to one CSV file
def filterWord(filename, directory, eebofolder,word):
    count = 0 
    dates = []
    outFileName = directory + "\\" + filename
    outFile = pd.DataFrame(columns=('title','author','publisher','date','text'))
    for cleanFile in os.listdir(eebofolder):
        #open and read each CSV 
        readFile = pd.read_csv(eebofolder + "\\" + cleanFile) 
        #read the entries of each row (each row is its own text)
        for i in range(len(readFile.index)):
            row = readFile[i:(i+1)]
            #convert the dates into the same format and exclude non-dated texts 
            if isinstance(row.iloc[0].date,str):
                date = (row.iloc[0].date)
            else:
                date = (row.iloc[0].date).astype(str)
            if ((date !='Date Not Found') & (date !='[\'Date Not Found\']')):
                if (date and date.strip()):
                    datenum = re.search('\d{4}', date)  
                    if (datenum != None):
                        text = row.iloc[0].text
                        if (re.search(word, text) != None):
                            date = int(str(datenum.group(0)))
                            dates.append(date)          
                            d = pd.DataFrame([(row.iloc[0].title,
                                                row.iloc[0].author,
                                                row.iloc[0].publisher,
                                                date,
                                                row.iloc[0].text)],
                                                columns=('title','author','publisher','date','text'))
                            outFile = outFile.append(d,ignore_index=True)
            count+=1
            if (count % 10000 == 0):
                print(count)
    outFile.to_csv(outFileName)

eic = re.compile("east india company|east-india company|east-india-company")
filterWord("dated_eic", directory, eebofolder, eic)